In [2]:
# import all necessary references
import pandas as pd
import importlib.metadata
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import yaml
from pprint import pprint
from math import floor
from pathlib import PurePath, Path
from kaggle_hcdr.utils import get_dir_path
from sklearn.model_selection import train_test_split


print(

    f"""
Pandas version: {pd.__version__}
Matplotlib Pyplot version: {importlib.metadata.version("matplotlib")}
Seaborn version: {sns.__version__}
PyYAML version: {yaml.__version__}    
    """
)


Pandas version: 2.0.0
Matplotlib Pyplot version: 3.7.1
Seaborn version: 0.12.2
PyYAML version: 6.0    
    


In [3]:
# Pandas configuration
pd.set_option("display.max_rows", 150)
pd.set_option("display.max_columns", 150)
pd.set_option("display.max_colwidth", None)

# Plot configuration
%matplotlib widget

# Path configuration
current_directory = Path.cwd()
current_directory

PosixPath('/home/xlevb/projects/kaggle-home-credit-default-risk/notebooks/iteration_1')

In [4]:
# Create references for data paths
data_directory = get_dir_path("data")

raw_application_test_file = data_directory.joinpath("01_raw", "application_test.csv")
raw_application_train_file = data_directory.joinpath("01_raw", "application_train.csv")
conf_directory = get_dir_path("conf")
project_parameters_file = conf_directory.joinpath("base", "parameters.yml")

In [5]:
with open(project_parameters_file, "r") as file:
    project_parameters = yaml.safe_load(file)

pprint(project_parameters)

{'dataset_variables_types': {'application_test': {'categorical_nominal': ['CODE_GENDER',
                                                                          'FLAG_WORK_PHONE',
                                                                          'FLAG_EMP_PHONE',
                                                                          'NAME_CONTRACT_TYPE',
                                                                          'FLAG_OWN_REALTY',
                                                                          'WEEKDAY_APPR_PROCESS_START',
                                                                          'FONDKAPREMONT_MODE',
                                                                          'FLAG_EMAIL',
                                                                          'WALLSMATERIAL_MODE',
                                                                          'FLAG_OWN_CAR',
                                                               

In [6]:
raw_application_train_df = pd.read_csv(
    filepath_or_buffer=raw_application_train_file,
    index_col="SK_ID_CURR",
    parse_dates=True,
    engine="pyarrow",
    # dtype_backend="pyarrow",
    encoding='unicode_escape'
)
raw_application_test_df = pd.read_csv(
    filepath_or_buffer=raw_application_test_file,
    index_col="SK_ID_CURR",
    parse_dates=True,
    engine="pyarrow",
    # dtype_backend="pyarrow",
    encoding='unicode_escape'
)

In [7]:
pre_feat_application_w_target = raw_application_train_df[project_parameters.get("target")].copy()
pre_feat_application_wo_target = raw_application_train_df.drop(columns=project_parameters.get("target"))

In [8]:
(
    pre_feat_application_wo_target_train,
    pre_feat_application_wo_target_test,
    pre_feat_application_w_target_train,
    pre_feat_application_w_target_test,
) = train_test_split(
    pre_feat_application_wo_target,
    pre_feat_application_w_target,
    test_size=0.25,
    random_state=42,
    shuffle=True
)

In [9]:
print(f"""
Training dataset input variables: {pre_feat_application_wo_target_train.shape}
Training dataset output variables: {pre_feat_application_w_target_train.shape}
Test dataset input variables: {pre_feat_application_wo_target_test.shape}
Test dataset output variables: {pre_feat_application_w_target_test.shape}
""")


Training dataset input variables: (230633, 120)
Training dataset output variables: (230633, 1)
Test dataset input variables: (76878, 120)
Test dataset output variables: (76878, 1)



In [52]:
categorical_columns_str = pre_feat_application_wo_target_train[project_parameters["dataset_variables_types"]["application_train"]["categorical_nominal"]].select_dtypes(include='object').columns.to_list()
categorical_columns_str.append("ORGANIZATION_TYPE")
categorical_columns_str.append("OCCUPATION_TYPE")
categorical_columns_str                          

['CODE_GENDER',
 'NAME_CONTRACT_TYPE',
 'FLAG_OWN_REALTY',
 'WEEKDAY_APPR_PROCESS_START',
 'FONDKAPREMONT_MODE',
 'WALLSMATERIAL_MODE',
 'FLAG_OWN_CAR',
 'NAME_INCOME_TYPE',
 'HOUSETYPE_MODE',
 'NAME_HOUSING_TYPE',
 'NAME_EDUCATION_TYPE',
 'EMERGENCYSTATE_MODE',
 'NAME_FAMILY_STATUS',
 'NAME_TYPE_SUITE',
 'ORGANIZATION_TYPE',
 'OCCUPATION_TYPE']

In [53]:
# Transform catgorical variables
feat_application_wo_target_train = pre_feat_application_wo_target_train.copy()
feat_application_wo_target_train = pd.get_dummies(
    feat_application_wo_target_train, columns=categorical_columns_str, dtype="int"
)

In [54]:
feat_application_wo_target_train

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,...,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Cleaning,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Culture,ORGANIZATION_TYPE_Electricity,ORGANIZATION_TYPE_Emergency,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Hotel,ORGANIZATION_TYPE_Housing,ORGANIZATION_TYPE_Industry: type 1,ORGANIZATION_TYPE_Industry: type 10,ORGANIZATION_TYPE_Industry: type 11,ORGANIZATION_TYPE_Industry: type 12,ORGANIZATION_TYPE_Industry: type 13,ORGANIZATION_TYPE_Industry: type 2,ORGANIZATION_TYPE_Industry: type 3,ORGANIZATION_TYPE_Industry: type 4,ORGANIZATION_TYPE_Industry: type 5,ORGANIZATION_TYPE_Industry: type 6,ORGANIZATION_TYPE_Industry: type 7,ORGANIZATION_TYPE_Industry: type 8,ORGANIZATION_TYPE_Industry: type 9,ORGANIZATION_TYPE_Insurance,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Legal Services,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Mobile,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Postal,ORGANIZATION_TYPE_Realtor,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA,OCCUPATION_TYPE_,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
400186,0,270000.0,539100.0,27652.5,450000.0,0.010147,-19802,-6322,-5503.0,-3336,NaN,1,1,1,1,0,0,1.0,2,2,15,0,0,0,0,0,0,0.572197,0.575546,0.165407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [55]:
feat_application_wo_target_test = pre_feat_application_wo_target_test.copy()
feat_application_wo_target_test = pd.get_dummies(
    feat_application_wo_target_test, columns=categorical_columns_str, dtype="int"
)

In [56]:
feat_application_wo_target_test

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,...,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Cleaning,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Culture,ORGANIZATION_TYPE_Electricity,ORGANIZATION_TYPE_Emergency,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Hotel,ORGANIZATION_TYPE_Housing,ORGANIZATION_TYPE_Industry: type 1,ORGANIZATION_TYPE_Industry: type 10,ORGANIZATION_TYPE_Industry: type 11,ORGANIZATION_TYPE_Industry: type 12,ORGANIZATION_TYPE_Industry: type 13,ORGANIZATION_TYPE_Industry: type 2,ORGANIZATION_TYPE_Industry: type 3,ORGANIZATION_TYPE_Industry: type 4,ORGANIZATION_TYPE_Industry: type 5,ORGANIZATION_TYPE_Industry: type 6,ORGANIZATION_TYPE_Industry: type 7,ORGANIZATION_TYPE_Industry: type 8,ORGANIZATION_TYPE_Industry: type 9,ORGANIZATION_TYPE_Insurance,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Legal Services,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Mobile,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Postal,ORGANIZATION_TYPE_Realtor,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA,OCCUPATION_TYPE_,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
384575,2,207000.0,465457.5,52641.0,418500.0,0.009630,-13297,-762,-637.0,-4307,19.0,1,1,0,1,0,0,4.0,2,2,11,0,0,0,0,1,1,0.675878,0.604894,0.000527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [57]:
feat_application_wo_target_train.isna().sum().sort_values(ascending=False)

COMMONAREA_MEDI                         161235
COMMONAREA_MODE                         161235
COMMONAREA_AVG                          161235
NONLIVINGAPARTMENTS_AVG                 160189
NONLIVINGAPARTMENTS_MEDI                160189
                                         ...  
WALLSMATERIAL_MODE_Monolithic                0
WALLSMATERIAL_MODE_Others                    0
WALLSMATERIAL_MODE_Panel                     0
WALLSMATERIAL_MODE_Stone, brick              0
OCCUPATION_TYPE_Waiters/barmen staff         0
Length: 250, dtype: int64

In [58]:
feat_application_wo_target_test.isna().sum().sort_values(ascending=False)

COMMONAREA_AVG                          53630
COMMONAREA_MEDI                         53630
COMMONAREA_MODE                         53630
NONLIVINGAPARTMENTS_MEDI                53325
NONLIVINGAPARTMENTS_AVG                 53325
                                        ...  
AMT_INCOME_TOTAL                            0
WALLSMATERIAL_MODE_Monolithic               0
WALLSMATERIAL_MODE_Others                   0
WALLSMATERIAL_MODE_Panel                    0
OCCUPATION_TYPE_Waiters/barmen staff        0
Length: 250, dtype: int64

In [59]:
feat_application_wo_target_train.fillna(value=0.0, inplace=True)

In [60]:
feat_application_wo_target_test.fillna(value=0.0, inplace=True)

In [61]:
feat_application_wo_target_train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 230633 entries, 400186 to 241394
Data columns (total 250 columns):
 #    Column                                             Dtype  
---   ------                                             -----  
 0    CNT_CHILDREN                                       int64  
 1    AMT_INCOME_TOTAL                                   float64
 2    AMT_CREDIT                                         float64
 3    AMT_ANNUITY                                        float64
 4    AMT_GOODS_PRICE                                    float64
 5    REGION_POPULATION_RELATIVE                         float64
 6    DAYS_BIRTH                                         int64  
 7    DAYS_EMPLOYED                                      int64  
 8    DAYS_REGISTRATION                                  float64
 9    DAYS_ID_PUBLISH                                    int64  
 10   OWN_CAR_AGE                                        float64
 11   FLAG_MOBIL                           

In [62]:
data_model_input = data_directory.joinpath("05_model_input")

application_train_wo_target_file = data_model_input.joinpath("application_train_wo_target.parquet")
application_test_wo_target_file = data_model_input.joinpath("application_test_wo_target_file.parquet")
application_train_w_target_file = data_model_input.joinpath("application_train_w_target_file.parquet")
application_test_w_target_file = data_model_input.joinpath("application_test_w_target_file.parquet")

In [63]:
feat_application_wo_target_train.to_parquet(application_train_wo_target_file)
feat_application_wo_target_test.to_parquet(application_test_wo_target_file)
pre_feat_application_w_target_train.to_parquet(application_train_w_target_file)
pre_feat_application_w_target_test.to_parquet(application_test_w_target_file)

In [64]:
project_parameters["features"] = feat_application_wo_target_train.columns.to_list()

In [65]:
with open(project_parameters_file, "w") as file:
    yaml.safe_dump(project_parameters, file)

In [66]:
with open(project_parameters_file, "r") as file:
    project_parameters_updated = yaml.safe_load(file)

project_parameters_updated

{'dataset_variables_types': {'application_test': {'categorical_nominal': ['CODE_GENDER',
    'FLAG_WORK_PHONE',
    'FLAG_EMP_PHONE',
    'NAME_CONTRACT_TYPE',
    'FLAG_OWN_REALTY',
    'WEEKDAY_APPR_PROCESS_START',
    'FONDKAPREMONT_MODE',
    'FLAG_EMAIL',
    'WALLSMATERIAL_MODE',
    'FLAG_OWN_CAR',
    'FLAG_MOBIL',
    'FLAG_PHONE',
    'NAME_INCOME_TYPE',
    'HOUSETYPE_MODE',
    'NAME_HOUSING_TYPE',
    'FLAG_CONT_MOBILE',
    'NAME_EDUCATION_TYPE',
    'EMERGENCYSTATE_MODE',
    'NAME_FAMILY_STATUS',
    'NAME_TYPE_SUITE'],
   'categorical_ordinal': ['REGION_RATING_CLIENT',
    'REGION_RATING_CLIENT_W_CITY'],
   'datetime': ['HOUR_APPR_PROCESS_START'],
   'numerical_continuous': ['EXT_SOURCE_3',
    'ENTRANCES_MODE',
    'BASEMENTAREA_MODE',
    'APARTMENTS_MEDI',
    'YEARS_BUILD_MEDI',
    'ENTRANCES_MEDI',
    'NONLIVINGAREA_MODE',
    'YEARS_BEGINEXPLUATATION_MODE',
    'ELEVATORS_MEDI',
    'YEARS_BEGINEXPLUATATION_MEDI',
    'AMT_INCOME_TOTAL',
    'NONLIVINGAPARTMENT